In [1]:
import dataset

In [2]:
regenerate=False
statements = dataset.load_statements(regenerate=regenerate)
statements_by_uid = { s.uid:s for s in statements }

In [3]:
keyword_counts = dataset.get_keyword_counts_from_statements(statements)
len(keyword_counts)

4546

In [4]:
import networkx as nx
G = nx.Graph()

In [5]:
#for i, s in enumerate(statements):
#    print(s)
#    if i>10: break
G.add_nodes_from(s.uid for s in statements)
len(G)

12039

In [6]:
G.add_nodes_from(k for k,c in keyword_counts.items())
len(G)

16585

In [313]:
# https://networkx.github.io/documentation/stable/reference/classes/generated/
#   networkx.Graph.add_weighted_edges_from.html
for i, s in enumerate(statements):
    G.add_weighted_edges_from(
        #(k, s.uid, 1.0) for k in s.keywords  # Evenly weighted :: Poor
        #(k, s.uid, 1.0+i) for k in s.keywords  # Demonstrates that weightings are 'weight'
        #(k, s.uid, 2.0-1.0/keyword_counts[k]) for k in s.keywords  # weight by how frequent keywords are
        #(k, s.uid, 3.0-1.0/len(s.keywords)-1.0/keyword_counts[k]) for k in s.keywords   # bit of both
        # Decent:
        (k, s.uid, 2.0-1.0/len(s.keywords)) for k in s.keywords   # weight by how many keywords the statement has
    )
len(G.edges)

51556

In [314]:
# Effectively kill some nodes
for node in ['be']:
    G.add_weighted_edges_from((node, n, 100.) for n in G.neighbors(node) )        

In [315]:
#[n for n in G.neighbors('iron')]
[n for n in G.neighbors('2ead-9402-4803-38bc')]

['more', 'dense', 'water', 'iron']

In [316]:
nx.dijkstra_path(G, 'iron', 'water')

['iron', '2ead-9402-4803-38bc', 'water']

In [317]:
for p in nx.all_shortest_paths(G, 'iron', 'water', weight='weight'): 
    print(p)

['iron', '2ead-9402-4803-38bc', 'water']


In [318]:
def explain_path(p):
    for i,n in enumerate(p):
        #if i%2>1:
        if len(n)==19:  # a uid
            print("   > "+n+" : "+statements_by_uid[n].raw_txt)
        else:
            print(n)
    print()

explain_path( nx.dijkstra_path(G, 'rust', 'water') )

rust
   > 6d3e-e79a-9d08-519b : iron in contact with water and oxygen will rust
water



In [319]:
for p in nx.all_shortest_paths(G, 'saltwater', 'reason', weight='weight'): 
    explain_path( p )

saltwater
   > d60d-6c10-0203-e659 : saltwater is a kind of substance
substance
   > 2a76-307e-23d8-5e78 : bringing substances to the body is a kind of function
bring
   > 1ced-cf2e-c041-478c : to cause is similar to bring
cause
   > ac5e-7fdf-8cec-126d : a cause of something is a reason for that something
reason

saltwater
   > d60d-6c10-0203-e659 : saltwater is a kind of substance
substance
   > 2a76-307e-23d8-5e78 : bringing substances to the body is a kind of function
function
   > da58-addb-0630-0b89 : causing is a kind of function
cause
   > ac5e-7fdf-8cec-126d : a cause of something is a reason for that something
reason

saltwater
   > d60d-6c10-0203-e659 : saltwater is a kind of substance
substance
   > bdf8-fbda-ff94-54f5 : a hot substance is a source of heat
source
   > 4bc3-00d6-bc97-8838 : to be a source of something means to cause that something
cause
   > ac5e-7fdf-8cec-126d : a cause of something is a reason for that something
reason

saltwater
   > d60d-6c10-0203-e659 :

In [320]:
regenerate=False
#qanda_train = dataset.load_qanda('train', regenerate=regenerate) # 1.8MB
qanda_dev   = dataset.load_qanda('dev', regenerate=regenerate)   # 400k in 496 lines
#qanda_test  = dataset.load_qanda('test', regenerate=regenerate)  # 800k

In [321]:
preds_baseline=dict() # qa_id -> [statements in order]
with open('/tmp/scorer/predict.txt', 'rt') as f:
    for l in f.readlines():
        qid, uid = l.strip().split('\t')
        if qid not in preds_baseline: preds_baseline[qid]=[]
        preds_baseline[qid].append(uid)

In [322]:
qa = qanda_dev[485]  #212  #4=astronomy

print(qa.question_id, qa.flags)
print("Q: "+qa.question.raw_txt)
print("  ", qa.question.keywords)
for i,ans in enumerate(qa.answers):
    print(f"A{i:1d}: "+ans.raw_txt)
    print("  ", ans.keywords)
print("explanation_gold:")
for ex in qa.explanation_gold:
    print(f"  {ex.uid} : {statements_by_uid[ex.uid].raw_txt}")
    print(f"{' '*25} : {statements_by_uid[ex.uid].keywords}")

print()
print(f"""baseline MAP = {dataset.silent_average_precision_score(
    set(e.uid for e in qa.explanation_gold), preds_baseline[qa.question_id][:]):.4f}""")

MCAS_2003_8_25 success
Q: In an automobile, which of the following components is designed specifically to give feedback to the driver?
   {'automobile', 'specifically', 'design', 'feedback', 'driver', 'give', 'follow', 'component'}
A0: speedometer
   {'speedometer'}
A1: steering wheel
   {'wheel', 'steering'}
A2: brake pedal
   {'pedal', 'brake'}
A3: car key
   {'key', 'car'}
explanation_gold:
  0a7c-d9fa-1622-6881 : a speedometer is used for giving a driver feedback on the speed of their vehicle
                          : {'speedometer', 'feedback', 'speed', 'driver', 'give', 'vehicle'}
  7cb7-40ac-f4d5-eb6c : an automobile is a kind of vehicle
                          : {'automobile', 'vehicle'}
  9948-f6ec-f997-4905 : a speedometer is a part of an automobile
                          : {'part', 'automobile', 'speedometer'}
  10d3-297c-e0e5-8531 : a component means a part of
                          : {'part', 'component'}

baseline MAP = 0.8929


In [323]:
statements_by_uid[qa.explanation_gold[2].uid].keywords # .add('part')

{'automobile', 'part', 'speedometer'}

In [324]:
','.join(qa.question.keywords), ','.join(qa.answers[0].keywords)

('automobile,specifically,design,feedback,driver,give,follow,component',
 'speedometer')

In [325]:
kw_all = qa.question.keywords | qa.answers[0].keywords
kw_all = set([kw for kw in kw_all if kw in G.nodes]) # Only use those in the vocab ...
','.join(kw_all)

'automobile,design,speedometer,feedback,driver,give,follow,component'

In [326]:
target='core'
#kw_all-set([target])

In [327]:
#d,p=nx.multi_source_dijkstra(G, kw_all-set([target]), target=target, weight='weight')
d,p=nx.multi_source_dijkstra(G, ['component'], target='speedometer', weight='weight')
#print(p)
explain_path( p ) # This is not great...

component
   > 7743-074e-340e-e2e9 : a component of something means a part of that something
part
   > 9948-f6ec-f997-4905 : a speedometer is a part of an automobile
speedometer



In [328]:
#kw_arr = list(kw_all)
#srcs, tgts, limit_i_j = kw_all, kw_all, True
srcs, tgts, limit_i_j = qa.question.keywords, qa.answers[0].keywords, False

srcs = set([n for n in srcs if n in G.nodes]) # Only use those in the vocab ...
tgts = set([n for n in tgts if n in G.nodes]) # Only use those in the vocab ...

node_cnt=dict()
for i, src in enumerate(srcs):
    print(f"{i}/{len(srcs)}")
    for j, tgt in enumerate(tgts):
        if limit_i_j and i>=j: continue
        for p in nx.all_shortest_paths(G, src, tgt, weight='weight'): 
            explain_path( p )
            for k,n in enumerate(p):
                if k%2==0:continue
                if n not in node_cnt: node_cnt[n]=0
                node_cnt[n]+=1
len(node_cnt)

0/7
automobile
   > 9948-f6ec-f997-4905 : a speedometer is a part of an automobile
speedometer

1/7
design
   > f885-fb7a-bfbb-9858 : a computer-aided design program can be used to draw three-dimensional objects with exact dimensions and angles
object
   > 21c4-0d4f-7e3c-12f6 : an object is made of its parts
part
   > 9948-f6ec-f997-4905 : a speedometer is a part of an automobile
speedometer

2/7
feedback
   > 0a7c-d9fa-1622-6881 : a speedometer is used for giving a driver feedback on the speed of their vehicle
speedometer

3/7
driver
   > 0a7c-d9fa-1622-6881 : a speedometer is used for giving a driver feedback on the speed of their vehicle
speedometer

4/7
give
   > 0a7c-d9fa-1622-6881 : a speedometer is used for giving a driver feedback on the speed of their vehicle
speedometer

5/7
follow
   > b15f-4dc7-7fdd-4f51 : In the water cycle , transpiration always follows plant uptake
plant
   > ed05-7140-7262-f394 : plant parts are made of plants
part
   > 9948-f6ec-f997-4905 : a speedomet

8

In [329]:
nodes_desc = sorted(node_cnt.items(),key=lambda n:-n[1])
#nodes_desc[:10]
len(nodes_desc)

8

In [330]:
[ f"{cnt:5d} : {uid:s} : {statements_by_uid[uid].raw_txt}" for uid,cnt in nodes_desc ][:10]

['    5 : 9948-f6ec-f997-4905 : a speedometer is a part of an automobile',
 '    3 : 0a7c-d9fa-1622-6881 : a speedometer is used for giving a driver feedback on the speed of their vehicle',
 '    1 : f885-fb7a-bfbb-9858 : a computer-aided design program can be used to draw three-dimensional objects with exact dimensions and angles',
 '    1 : 21c4-0d4f-7e3c-12f6 : an object is made of its parts',
 '    1 : b15f-4dc7-7fdd-4f51 : In the water cycle , transpiration always follows plant uptake',
 '    1 : ed05-7140-7262-f394 : plant parts are made of plants',
 '    1 : 7743-074e-340e-e2e9 : a component of something means a part of that something',
 '    1 : 10d3-297c-e0e5-8531 : a component means a part of']

In [331]:
gold_ex_uid = set(e.uid for e in qa.explanation_gold)
preds_uid   = preds_baseline[qa.question_id]

def base_uids_list(arr):
    base_arr=[]
    for a in arr: 
        base=a[:19]
        if base not in base_arr: base_arr.append(base)
    return base_arr

nodes_uid   = base_uids_list( [uid for uid,_ in nodes_desc] )

for limit in [10000, 64, 32,16] :
    sc_baseline= dataset.silent_average_precision_score(
                   gold_ex_uid, preds_uid[:limit])
    sc_graph   = dataset.silent_average_precision_score(
                   gold_ex_uid, nodes_uid[:limit])
    print(f"{limit:5d} : baseline={sc_baseline:.4f}, graph={sc_graph:.4f}")
print(f"recall:          {len(gold_ex_uid & set(preds_uid[:64]))/len(gold_ex_uid):.4f}"+
               f"        {len(gold_ex_uid & set(nodes_uid[:64]))/len(gold_ex_uid):.4f} ")

10000 : baseline=0.8929, graph=0.5938
   64 : baseline=0.8929, graph=0.5938
   32 : baseline=0.8929, graph=0.5938
   16 : baseline=0.8929, graph=0.5938
recall:          1.0000        0.7500 


In [332]:
# Agrees with baseline scoring... (good job too, since it's the actual scorer)
# python ../tg2020task/evaluate.py --gold ../tg2020task/questions.dev.tsv /tmp/scorer/predict.txt 

In [333]:
def get_min_distance_statements(qa, all_to_all=False):
    if all_to_all:
        kw_all = qa.question.keywords | qa.answers[0].keywords
        #kw_all = set([kw for kw in kw_all if kw in G.nodes]) # Only use those in the vocab ...
        #print(len(kw_all))
        srcs, tgts, limit_i_j = kw_all, kw_all, True
    else:
        srcs, tgts, limit_i_j = qa.question.keywords, qa.answers[0].keywords, False

    srcs = set([n for n in srcs if n in G.nodes]) # Only use those in the vocab ...
    tgts = set([n for n in tgts if n in G.nodes]) # Only use those in the vocab ...
        
    node_cnt=dict()
    for i, src in enumerate(srcs):
        #print(f"{i}/{len(srcs)}")
        for j, tgt in enumerate(tgts):
            if limit_i_j and i>=j: continue
            try:
                paths = nx.all_shortest_paths(G, src, tgt, weight='weight')
                for p in paths: 
                    #explain_path( p )
                    for k,n in enumerate(p):
                        #if k%2==0:continue
                        if len(n)<19:continue  # Ignore the keywords
                        if n not in node_cnt: node_cnt[n]=0
                        node_cnt[n]+=1
            except:
                print("  Cannot get to "+tgt)
    nodes_desc = sorted(node_cnt.items(),key=lambda n:-n[1])
    #print(nodes_desc)
    nodes_uid   = base_uids_list( [uid for uid,_ in nodes_desc] )
    return nodes_uid


#qa = qanda_dev[4]  #212  #4=astronomy    
if False:
    #with open("/tmp/scorer/predict_graph.txt", "wt") as f:
    with open("../predictions/predict_graph-q-to-a.txt", "wt") as f:
        for qa_i, qa in enumerate(qanda_dev):
            print(f"Running : {qa_i}")
            uids = get_min_distance_statements(qa)  #, all_to_all=True) 
            for uid in uids:
                f.write(f"{qa.question_id}\t{uid}\n")
#get_min_distance_statements(qanda_dev[486])

In [334]:
# python ../tg2020task/evaluate.py --gold ../tg2020task/questions.dev.tsv /tmp/scorer/predict_graph.txt


In [335]:
for t in dataset.nlp('if a substance has a higher density than another substance , then the molecules in the substance will be closer than those of the other substance'):
#for t in dataset.nlp('About how many times does the moon orbit Earth in a year?'):
#for t in dataset.nlp('approximately means about'):
    print(t.pos_, t.text, t.lemma_)
# ADP About about ...

SCONJ if if
DET a a
NOUN substance substance
AUX has have
DET a a
ADJ higher high
NOUN density density
SCONJ than than
DET another another
NOUN substance substance
PUNCT , ,
ADV then then
DET the the
NOUN molecules molecule
ADP in in
DET the the
NOUN substance substance
VERB will will
AUX be be
ADJ closer close
SCONJ than than
DET those those
ADP of of
DET the the
ADJ other other
NOUN substance substance


In [336]:
# Different idea : let's create a new graph from the QuestionAnswer and the known explanation_gold nodes
#   Then : Have a look at nodes that only have 1 edge

In [337]:
# https://networkx.github.io/documentation/stable/reference/classes/generated/networkx.Graph.subgraph.html

In [338]:
kw_all = qa.question.keywords | qa.answers[0].keywords
copy_qa = set([kw for kw in kw_all if kw in G.nodes]) # Only use those in the vocab ...
len(copy_qa), ','.join(copy_qa)

(8, 'automobile,design,speedometer,feedback,driver,give,follow,component')

In [339]:
copy_gold = set([e.uid for e in qa.explanation_gold])
len(copy_gold), ','.join(copy_gold)

(4,
 '9948-f6ec-f997-4905,10d3-297c-e0e5-8531,7cb7-40ac-f4d5-eb6c,0a7c-d9fa-1622-6881')

In [340]:
for c in copy_gold:
    print(c, list(G.neighbors(c)))

9948-f6ec-f997-4905 ['automobile', 'speedometer', 'part']
10d3-297c-e0e5-8531 ['part', 'component']
7cb7-40ac-f4d5-eb6c ['automobile', 'vehicle']
0a7c-d9fa-1622-6881 ['speedometer', 'feedback', 'speed', 'driver', 'give', 'vehicle']


In [341]:
copy_adj = set([nbr for n in copy_gold for nbr in G.neighbors(n) ]) # Order of for loops v important
len(copy_adj), ','.join(sorted(copy_adj))

(9, 'automobile,component,driver,feedback,give,part,speed,speedometer,vehicle')

In [342]:
#copy_nodes = set([n for lst in [copy_gold, copy_adj, copy_qa] for n in lst])
#len(copy_nodes), ','.join(sorted(copy_nodes))

In [343]:
SG = G.__class__()
SG.add_nodes_from((n, G.nodes[n]) for n in copy_gold )   
SG.add_nodes_from((n, G.nodes[n]) for n in copy_adj )   
SG.add_nodes_from((n, G.nodes[n]) for n in copy_qa )   
SG.add_edges_from((n, nbr, d)
    for n, nbrs in G.adj.items() if n in copy_gold
    for nbr, d in nbrs.items() if nbr in copy_adj)
SG.add_edges_from((n, nbr, d)
    for n, nbrs in G.adj.items() if n in copy_gold
    for nbr, d in nbrs.items() if nbr in copy_qa)
len(SG)

15

In [344]:
# Now for the nodes with only one edge:
for n in SG.nodes:
    l=len(list(SG.neighbors(n)))
    if l<=1:
        print(n, l)

feedback 1
speed 1
driver 1
give 1
component 1
design 0
follow 0


In [345]:
def get_lonely_nodes(ex_list, copy_qa=copy_qa, ):
    copy_adj = set([nbr for n in ex_list for nbr in G.neighbors(n) ]) # Order of for loops v important
    
    SG = G.__class__()
    SG.add_nodes_from((n, G.nodes[n]) for n in ex_list )   
    SG.add_nodes_from((n, G.nodes[n]) for n in copy_adj )   
    SG.add_nodes_from((n, G.nodes[n]) for n in copy_qa )   
    SG.add_edges_from((n, nbr, d)
        for n, nbrs in G.adj.items() if n in ex_list
        for nbr, d in nbrs.items() if nbr in copy_adj)
    SG.add_edges_from((n, nbr, d)
        for n, nbrs in G.adj.items() if n in ex_list
        for nbr, d in nbrs.items() if nbr in copy_qa)
    #print(len(SG))
    print("question keywords", [n for n in (qa.question.keywords) if n in SG.nodes])
    print("answers[0] keywords", [n for n in (qa.answers[0].keywords) if n in SG.nodes])
    print("leaf nodes", [n for n in (SG.nodes - copy_qa) if len(list(SG.neighbors(n)))<=1 ])
    
    # qa.question.keywords | qa.answers[0].keywords
    empty_q = [n for n in (qa.question.keywords)
                if n in SG.nodes and len(list(SG.neighbors(n)))==0 ]
    empty_a = [n for n in (qa.answers[0].keywords)
                if n in SG.nodes and len(list(SG.neighbors(n)))==0 ]
    lonely_leaf = set( [n for n in (SG.nodes - copy_qa)
                    if len(list(SG.neighbors(n)))<=1 ] )
    
    # Links between explanation statements
    # == non-qa and non-leaf words?
    #  ?+? multi-linked qa words?
    statement_links = [n for n in (SG.nodes - copy_qa - lonely_leaf)
                                if len(n)!=19 ]
    
    return empty_q, empty_a, lonely_leaf, statement_links

get_lonely_nodes(copy_gold)

question keywords ['automobile', 'design', 'feedback', 'driver', 'give', 'follow', 'component']
answers[0] keywords ['speedometer']
leaf nodes ['speed']


(['design', 'follow'], [], {'speed'}, ['part', 'vehicle'])

In [346]:
for uid in sorted(copy_gold):
    s = statements_by_uid[uid]
    print(f"  {uid} : {s.raw_txt}")
    print(f"          {s.keywords}")

  0a7c-d9fa-1622-6881 : a speedometer is used for giving a driver feedback on the speed of their vehicle
          {'speedometer', 'feedback', 'speed', 'driver', 'give', 'vehicle'}
  10d3-297c-e0e5-8531 : a component means a part of
          {'part', 'component'}
  7cb7-40ac-f4d5-eb6c : an automobile is a kind of vehicle
          {'automobile', 'vehicle'}
  9948-f6ec-f997-4905 : a speedometer is a part of an automobile
          {'part', 'automobile', 'speedometer'}


In [353]:
# Now look at the predictions from baseline_retrieval
ex_guess = preds_baseline[qa.question_id][:5]
get_lonely_nodes( ex_guess )

question keywords ['automobile', 'design', 'feedback', 'driver', 'give', 'follow', 'component']
answers[0] keywords ['speedometer']
leaf nodes ['speed', 'source', 'vehicle']


(['design', 'follow'], [], {'source', 'speed', 'vehicle'}, ['part'])

In [352]:
for uid in ex_guess:
    s = statements_by_uid[uid]
    print(f"  {uid} : {s.raw_txt}")
    print(f"          {s.keywords}")

  0a7c-d9fa-1622-6881 : a speedometer is used for giving a driver feedback on the speed of their vehicle
          {'speedometer', 'feedback', 'speed', 'driver', 'give', 'vehicle'}
  9948-f6ec-f997-4905 : a speedometer is a part of an automobile
          {'part', 'automobile', 'speedometer'}
  10d3-297c-e0e5-8531 : a component means a part of
          {'part', 'component'}
  7743-074e-340e-e2e9 : a component of something means a part of that something
          {'part', 'component'}
  c273-104b-3c47-f637 : to give off means to be the source of
          {'source', 'give'}
  9749-fd22-c617-e673 : give off is similar to make
          {'make', 'give'}
  7cb7-40ac-f4d5-eb6c : an automobile is a kind of vehicle
          {'automobile', 'vehicle'}
  e112-616e-e25b-4a79 : a circuit is a kind of electrical component
          {'electrical', 'circuit', 'component'}
  eb14-f908-f76d-7cf0 : a car is a kind of automobile
          {'automobile', 'car'}
  f8ad-6ff2-e6fa-3dee : the automobile is 